In [351]:
import cudf as cd
import cupy as cp
import pandas as pd
import numpy as np
import seaborn as sns

In [352]:
dftrain1=cd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
dftest1=cd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [353]:
dftrain1.head()

In [354]:
#Preprocessing Training data
dftrain1.dtypes

In [355]:
dftrain1.info()

In [356]:
toprocess=[]
for i in dftrain1:
    if dftrain1[i].isnull().sum() > 0:
        toprocess.append(i)
toprocess

In [357]:
inttoprocess=dftrain1[toprocess].select_dtypes(include=['float64']).columns
cattoprocess=dftrain1[toprocess].select_dtypes(include=['object']).columns

In [358]:
for i in inttoprocess:
    dftrain1[i]=dftrain1[i].fillna(dftrain1[i].mean())

In [359]:
for i in cattoprocess:
    dftrain1[i]=dftrain1[i].fillna(method='ffill')

In [360]:
#No Null values in Training Data
toprocess1=[]
for i in dftrain1:
    if dftrain1[i].isnull().sum() > 0:
        toprocess1.append(i)
toprocess1

In [361]:
toprocesstest=[]
for i in dftest1:
    if dftest1[i].isnull().sum() > 0:
        toprocesstest.append(i)
toprocesstest

In [362]:
inttoprocesstest=dftest1[toprocesstest].select_dtypes(include=['float64']).columns
cattoprocesstest=dftest1[toprocesstest].select_dtypes(include=['object']).columns

In [363]:
for i in inttoprocesstest:
    dftest1[i]=dftest1[i].fillna(dftest1[i].mean())

In [364]:
for i in cattoprocesstest:
    dftest1[i]=dftest1[i].fillna(method='ffill')

In [365]:
#No Null Values now in Test Data
toprocess1test=[]
for i in dftest1:
    if dftest1[i].isnull().sum() > 0:
        toprocess1test.append(i)
toprocess1test

In [366]:
catdata=dftrain1.select_dtypes(include=['object']).columns
catdata

In [367]:
#Encoding Categorical Data in Training Data
dftrain1=cd.get_dummies(dftrain1,columns=catdata)
dftrain1

In [368]:
dftrain1.info()

In [369]:
#All Categorical Encoded Values in Training Data
catdat1a=dftrain1.select_dtypes(include=['object']).columns
catdat1a

In [370]:
#Encoding Categorical Values in Test Data
cattestdata=dftest1.select_dtypes(include=['object']).columns
dftest1=cd.get_dummies(dftest1,columns=cattestdata)
dftest1

In [371]:
#All Categorical Encoded Values in Test Data
catdata1test=dftest1.select_dtypes(include=['object']).columns
catdata1test

In [372]:
a=dftrain1.columns.intersection(dftest1.columns)
b=dftest1.columns.intersection(dftrain1.columns)
dftrain=dftrain1[a]
dftest=dftest1[b]
dftest=dftest.drop(['Id'],axis=1)
dftrain=dftrain.drop(['Id'],axis=1)
dftrain['SalePrice']=dftrain1['SalePrice']

In [373]:
dftest

In [374]:
#Dimensionality Reduction on df_train_common
# from sklearn.decomposition import PCA
# pca=PCA(n_components=200)
# pca.fit(df_train_common)

In [375]:
#Scaling Features
from cuml.preprocessing import StandardScaler
ss=StandardScaler()
sdftrain=ss.fit_transform(dftrain)

In [376]:
dftest.select_dtypes(include=['object']).columns

In [377]:
dftrain.columns

In [378]:
#Model Training on df_train
x=dftrain.drop('SalePrice',axis=1)
y=dftrain['SalePrice']

In [379]:
from cuml import LinearRegression
lr=LinearRegression(fit_intercept = True, normalize = False,algorithm = 'svd')
reg = lr.fit(x,y)
print("Coefficients:")
print(reg.coef_)
print("Intercept:")
print(reg.intercept_)

In [380]:
ypred=lr.predict(dftest)
ypred

In [381]:
idd=list(range(1461,2920))
ypredlist=ypred.to_arrow().to_pylist()
data={'Id':idd,'SalePrice':ypredlist}
resdf=cd.DataFrame(data)

In [382]:
resdf

In [384]:
resdf.to_csv('Submission10.csv',columns=['Id','SalePrice'],index=False)